# **SET UP**

In [ ]:
!pip install SciencePlots

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import scienceplots

from scipy import stats
from scipy.stats import t, linregress
from matplotlib.dates import date2num, num2date
import datetime
from datetime import datetime, date, timedelta
import folium
import branca.colormap
from geopy.geocoders import Nominatim

# **Newly updated MCL function**

In [ ]:
class DataAnalyzer:
    def __init__(self, data_or_url):
        if isinstance(data_or_url, str):  # Assuming file path is a string
            self.data_url = data_or_url
            self.data = self.load_data()
        elif isinstance(data_or_url, pd.DataFrame):  # Assuming DataFrame
            self.data = data_or_url
        else:
            raise ValueError("Input must be a file path or a pandas DataFrame")

    def load_data(self):
        file_extension = self.data_url.split('.')[-1]
        if file_extension == 'csv':
            return pd.read_csv(self.data_url)
        elif file_extension in ['xlsx', 'xlsm', 'xltx', 'xltm']:  # Add more excel file extensions if needed
            return pd.read_excel(self.data_url)
        else:
            raise ValueError("Unsupported file format")

    def print_column_names(self):
        print("Column names in the dataset:", self.data.columns.tolist())

    def preprocess_data(self, well_name_column, analyte_name_column, well_name, analyte_name):
        filtered_data = self.data[(self.data[well_name_column] == well_name) & (self.data[analyte_name_column] == analyte_name)]
        filtered_data = filtered_data.dropna(subset=['COLLECTION_DATE', 'RESULT'])
        filtered_data['COLLECTION_DATE'] = pd.to_datetime(filtered_data['COLLECTION_DATE']).dt.date
        filtered_data['RESULT_LOG'] = np.log10(filtered_data['RESULT'])
        return filtered_data

    def remove_outliers(self, data, result_column='RESULT_LOG', z_threshold=4):
        z = np.abs(stats.zscore(data[result_column]))
        return data[(z < z_threshold)]

    def predict_time_at_MCL(self, slope, intercept, log_MCL, earliest_date):
        if slope >= 0:
            return None
        time_at_MCL = (log_MCL - intercept) / slope
        try:
            time_at_MCL_date = num2date(time_at_MCL).replace(tzinfo=None).date()
            if earliest_date.year < 1 or time_at_MCL_date.year >= 10000 or time_at_MCL_date <= earliest_date:
                return None
            return time_at_MCL_date
        except ValueError:
            return None

    def adjust_date_based_on_confidence(self, base_date, slope, confidence_interval, days_per_unit):
        if base_date is None or slope == 0:
            return None, None, None
        days_adjustment = confidence_interval / abs(slope) * days_per_unit
        upper_date = base_date + timedelta(days=days_adjustment) if base_date else None
        lower_date = base_date - timedelta(days=days_adjustment) if base_date else None
        return lower_date, upper_date

    def generate_analyte_summary(self, analyte_name_column, analyte_name, well_name_column, MCL):
        unique_wells = self.data[self.data[analyte_name_column] == analyte_name][well_name_column].unique()
        summary = []

        for well in unique_wells:
            query = self.preprocess_data(well_name_column, analyte_name_column, well, analyte_name)
            if query.empty or len(query['COLLECTION_DATE'].unique()) < 2:
                continue

            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']
            slope, intercept, r_value, p_value, std_err = linregress(x, y)

            if slope > 0:
                trend = 'Increase'
            elif slope < 0:
                trend = 'Decrease'
            else:
                trend = 'Neutral'

            if trend == 'Neutral':
                continue

            log_MCL = np.log10(MCL)
            time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE']))

            # Calculate total decrease duration (for wells with decreasing trends)
            total_decrease_duration = None
            if trend == 'Decrease' and time_at_MCL:
                total_decrease_duration = (time_at_MCL - min(query['COLLECTION_DATE'])).days

            df = len(x) - 2
            ts = t.ppf(1 - 0.025, df)  # Two-sided t-critical value for 95% CI
            ci_slope = ts * std_err

            lower_date, upper_date = None, None
            if time_at_MCL and isinstance(time_at_MCL, date):
                lower_date, upper_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ci_slope, 365.25)

            summary.append({
                'well_name': well,
                'slope': slope,
                'trend': trend,
                'time@MCL': time_at_MCL,
                'lower_time@MCL': lower_date,
                'upper_time@MCL': upper_date,
                'total_decrease_duration': total_decrease_duration  # Add total decrease duration
            })

        summary_df = pd.DataFrame(summary)
        return summary_df

    def compile_results(self, well_analyte_pairs, well_name_column, analyte_name_column, MCL):
        results = []
        for well_name, analyte_name in well_analyte_pairs:
            time_at_MCL = None
            query = self.preprocess_data(well_name_column, analyte_name_column, well_name, analyte_name)
            if query.empty:
                continue

            query = self.remove_outliers(query)
            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']

            if len(x) < 2:
                continue

            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            df = len(x) - 2
            ts = t.ppf(1 - 0.025, df)

            trend = "Increase" if slope > 0 else "Decrease" if slope < 0 else "Neutral"
            log_MCL = np.log10(MCL) if MCL else None
            earliest_date = min(query['COLLECTION_DATE'])
            time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, earliest_date) if MCL else None

            # Calculate total decrease duration (for wells with decreasing trends)
            total_decrease_duration = None
            if trend == 'Decrease' and time_at_MCL:
                total_decrease_duration = (time_at_MCL - min(query['COLLECTION_DATE'])).days

            lower_date, upper_date = None, None
            if time_at_MCL and isinstance(time_at_MCL, date):
                lower_date, upper_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ts * std_err, 365.25)

            results.append({
                'well_name': well_name,
                'slope': slope,
                'trend': trend,
                'time@MCL': time_at_MCL,
                'lower_time@MCL': lower_date,
                'upper_time@MCL': upper_date,
                'total_decrease_duration': total_decrease_duration  # Add total decrease duration
            })

        results_df = pd.DataFrame(results)
        results_df.to_csv('time2MCL_results.csv', index=False)
        return results_df

    def plot_MCL(self, well_name_column, analyte_name_column, well_name, analyte_name, year_interval=20, save_dir='plot_MCL', MCL=None, extrapolate_until_year=2025):
        plt.style.use(['science', 'no-latex'])  # Using the SciencePlots package with 'science' style

        # Set the global font size parameters
        plt.rcParams.update({
            'font.size': 18,           # General font size
            'axes.titlesize': 18,      # Title font size
            'axes.labelsize': 18,      # Axis labels font size
            'xtick.labelsize': 18,     # X-tick label font size
            'ytick.labelsize': 18,     # Y-tick label font size
            'legend.fontsize': 18,     # Legend font size
            'figure.titlesize': 18     # Figure title font size
        })

        query = self.preprocess_data(well_name_column, analyte_name_column, well_name, analyte_name)
        if query.empty:
            print(f'No results found for {well_name} and {analyte_name}')
            return

        query = self.remove_outliers(query, result_column='RESULT_LOG')
        x = date2num(query['COLLECTION_DATE'])
        y = query['RESULT_LOG']

        if len(x) < 2:
            print("Not enough data for regression.")
            return

        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        df = len(x) - 2  # degrees of freedom for t-distribution
        ts = t.ppf(1 - 0.025, df)  # 95% confidence interval multiplier

        log_MCL = np.log10(MCL) if MCL else None
        time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE'])) if MCL and slope < 0 else 'Not available'

        lower_date, upper_date = None, None
        if time_at_MCL and isinstance(time_at_MCL, date):
            lower_date, upper_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ts * std_err, 365.25)

        # Prepare for extrapolation
        extrapolate_date = np.datetime64(f'{extrapolate_until_year}-12-31')
        x_extrapolate = np.linspace(x.min(), date2num(extrapolate_date), 100)
        y_extrapolate = slope * x_extrapolate + intercept

        # Ensure all plots are of the same size
        plt.figure(figsize=(10, 6))

        # Disable the grid mode
        plt.grid(False)

        # Only show the data point dots, no associated numbers
        plt.scatter(query['COLLECTION_DATE'], y, label='Data Points')
        plt.xlim([num2date(x.min()), num2date(x_extrapolate.max())])
        plt.ylim([0.0, max(y.max(), y_extrapolate.max())])
        plt.plot(num2date(x_extrapolate), y_extrapolate, 'r--', label=f'Extrapolated Regression (Decrease)')

        if log_MCL is not None:
            plt.axhline(y=log_MCL, color='green', linestyle='--', label=f'MCL (log10): {log_MCL:.2f}')

        # Adjusting position of the parameter box to avoid overlap with axis values
        plt.figtext(0.15, 0.02, f"Slope: {slope:.2e}\nTime@MCL: {time_at_MCL}\nLower CI: {lower_date}\nUpper CI: {upper_date}", fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

        plt.xlabel('Collection Date')
        plt.ylabel('Log-Concentration (pCi/mL)')
        plt.title(f'{well_name} - {analyte_name}')
        plt.legend()

        # Set a specific year interval for ticks so that around 10 ticks are shown
        plt.gca().xaxis.set_major_locator(mdates.YearLocator(base=year_interval))  # Adjust year interval (20 years in this case)
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gcf().autofmt_xdate()

        plt.xlim([num2date(x.min()), num2date(x_extrapolate.max())])

        figure_path = f'{save_dir}/{well_name}-{analyte_name}.png'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(figure_path)

        # Close the plot to avoid showing it multiple times
        # plt.close()

        return figure_path

    def generate_analysis_dataframe(self, analyte_name_column, well_name_column, analyte_name, MCL):
        analyte_summary_df = self.generate_analyte_summary(analyte_name_column, analyte_name, well_name_column, MCL)
        analysis_data = []

        for index, row in analyte_summary_df.iterrows():
            if row['trend'] == 'Neutral':
                continue

            plot_path = self.plot_MCL(well_name_column, analyte_name_column, row['well_name'], analyte_name, year_interval=5, save_dir='plot_MCL', MCL=MCL)

            analysis_data.append({
                'analyte_name': analyte_name,
                'well_name': row['well_name'],
                'slope': row['slope'],
                'trend': row['trend'],
                'time@MCL': row['time@MCL'],
                'lower_time@MCL': row['lower_time@MCL'],
                'upper_time@MCL': row['upper_time@MCL'],
                'total_decrease_duration': row['total_decrease_duration'],
                'path_to_figure': plot_path
            })

        return pd.DataFrame(analysis_data)

    def generate_decreasing_well_list(self, analyte_name_column, analyte_name, well_name_column, MCL):
        analyte_summary_df = self.generate_analyte_summary(analyte_name_column, analyte_name, well_name_column, MCL)
        decreasing_well_list = analyte_summary_df[analyte_summary_df['trend'] == 'Decrease']['well_name'].tolist()
        return decreasing_well_list

    def extract_location_data(self, well_name_column, decreasing_well_list):
        location_data = self.data[self.data[well_name_column].isin(decreasing_well_list)]
        return location_data

In [ ]:
data_analyzer = DataAnalyzer('https://raw.githubusercontent.com/ALTEMIS-DOE/pylenm/master/notebooks/data/FASB_Data_thru_3Q2015_Reduced_Demo.csv')
data_analyzer.print_column_names()  # This will print column names, helping you to identify the correct ones

# **Plot MCL for a specific well**

In [ ]:
data_analyzer.plot_MCL(
    well_name_column='STATION_ID',
    analyte_name_column='ANALYTE_NAME',
    well_name='FPZ  4A',
    analyte_name='IODINE-129',
    MCL=10,
    extrapolate_until_year=2050
)

# **Data Frame Table**

# ***Sr-90***

In [ ]:
analyte_summary_df_sr_90 = data_analyzer.generate_analyte_summary(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='STRONTIUM-90',
    well_name_column='STATION_ID',
    MCL=10
)
print(analyte_summary_df_sr_90)

# ***I-129***

In [ ]:
analyte_summary_df_i_129 = data_analyzer.generate_analyte_summary(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='IODINE-129',
    well_name_column='STATION_ID',
    MCL=10
)
print(analyte_summary_df_i_129)

# **Store Data Frame for a Specific Analyte**

# ***Sr-90***

In [ ]:
sr_90_data_frame = data_analyzer.generate_analysis_dataframe(
    'ANALYTE_NAME',
    'STATION_ID',
    'STRONTIUM-90',
    10)

# ***I-129***

In [ ]:
i_129_data_frame = data_analyzer.generate_analysis_dataframe(
    'ANALYTE_NAME',
    'STATION_ID',
    'IODINE-129',
    10)

# **Report**

In [ ]:
def report_of_data_frame(df, original_data, analyte_name):
    plt.style.use(['science', 'no-latex'])
    plt.grid(False)

    # Plot histogram
    df['total_decrease_duration'].hist(bins=20)
    plt.title('Histogram of Total Decrease Duration', fontsize=14) # Increase the font size
    plt.xlabel('Total Decrease Duration (days)', fontsize=12) # Increase the font size
    plt.ylabel('Frequency', fontsize=12)
    plt.show()

    # Key details
    total_decrease_duration = df['total_decrease_duration'].sum()
    print(f"Overall Total Decrease Duration: {total_decrease_duration} days")

    trend_counts = df['trend'].value_counts()
    print(f"\nTrend counts:\n{trend_counts}")

    avg_decrease_duration = df[df['trend'] == 'Decrease']['total_decrease_duration'].mean()
    print(f"Average Decrease Duration for Decreasing Wells: {avg_decrease_duration} days")

    all_wells_count = original_data[original_data['ANALYTE_NAME'] == analyte_name]['STATION_ID'].nunique()
    print(f"\nTotal number of wells in the original dataset for analyte '{analyte_name}': {all_wells_count}")

    decreasing_wells = df[df['trend'] == 'Decrease']
    wells_less_than_10_years = decreasing_wells[decreasing_wells['total_decrease_duration'] < 3650].shape[0]
    percentage_wells_less_than_10_years = (wells_less_than_10_years / decreasing_wells.shape[0]) * 100
    print(f"\nNumber of decreasing wells with time-to-MCL less than 10 years: {wells_less_than_10_years}")
    print(f"Percentage of decreasing wells with time-to-MCL less than 10 years: {percentage_wells_less_than_10_years:.2f}%")

    trends_per_class = df.groupby('well_name')['trend'].value_counts().unstack(fill_value=0)
    print(f"\nTrend counts per class:\n{trends_per_class}")

    overall_trend_counts = df['trend'].value_counts()
    decreasing_wells_count = overall_trend_counts.get('Decrease', 0)
    stable_wells_count = overall_trend_counts.get('Neutral', 0)
    increasing_wells_count = overall_trend_counts.get('Increase', 0)
    print(f"\nFor the whole analyte:\nDecreasing: {decreasing_wells_count}\nStable: {stable_wells_count}\nIncreasing: {increasing_wells_count}")

    median_duration = df['total_decrease_duration'].median()
    percentile_25 = np.percentile(df['total_decrease_duration'], 25)
    percentile_75 = np.percentile(df['total_decrease_duration'], 75)
    print(f"\nMedian Decrease Duration: {median_duration} days")
    print(f"25th Percentile Decrease Duration: {percentile_25} days")
    print(f"75th Percentile Decrease Duration: {percentile_75} days")

    skewness = stats.skew(df['total_decrease_duration'].dropna())
    print(f"Skewness of Decrease Duration Distribution: {skewness}")

    kurtosis = stats.kurtosis(df['total_decrease_duration'].dropna())
    print(f"Kurtosis of Decrease Duration Distribution: {kurtosis}")

    patterns = {
        'FSB**D': 'FSB\\d+D$',
        'FSB**DR': 'FSB\\d+DR$',
        'FSB Number Only': '^FSB\\d+$',
        'FPZ': '^FPZ'
    }

    for category, pattern in patterns.items():
        print(f"\nAnalysis for {category}:")
        category_well_count = original_data[original_data['STATION_ID'].str.contains(pattern, regex=True)]['STATION_ID'].nunique()
        print(f"Number of wells for {category}: {category_well_count}")

        category_df = df[df['well_name'].str.contains(pattern, regex=True)]

        category_trend_counts = category_df['trend'].value_counts()
        print(f"Trend counts for {category}:\n{category_trend_counts}")

        below_MCL_count = category_df[category_df['total_decrease_duration'] <= 0].shape[0]
        print(f"Wells already below MCL for {category}: {below_MCL_count}")

        if not category_df.empty:
            category_median_duration = category_df['total_decrease_duration'].median()
            category_skewness = stats.skew(category_df['total_decrease_duration'].dropna())
            print(f"Median Decrease Duration for {category}: {category_median_duration} days")
            print(f"Skewness of Decrease Duration Distribution for {category}: {category_skewness}")

In [ ]:
report_of_data_frame(sr_90_data_frame, data_analyzer.data, 'STRONTIUM-90')

In [ ]:
report_of_data_frame(i_129_data_frame, data_analyzer.data, 'IODINE-129')

In [ ]:
def compare_analytes_histogram(df1, df2, analyte1_name, analyte2_name):
    plt.style.use(['science', 'no-latex'])
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 12))  # Changed layout to vertical
    df1_filtered = df1['total_decrease_duration'].dropna()
    df2_filtered = df2['total_decrease_duration'].dropna()
    axes[0].grid(False)
    axes[1].grid(False)
    hist1, bins1 = np.histogram(df1_filtered, bins=20)
    hist2, bins2 = np.histogram(df2_filtered, bins=20)

    # Plot for analyte 1
    axes[0].hist(df1_filtered, bins=20)
    axes[0].set_title(f'(a)', fontsize=14)
    axes[0].set_xlabel('Total Decrease Duration (days)', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)

    # Plot for analyte 2
    axes[1].hist(df2_filtered, bins=20)
    axes[1].set_title(f'(b)', fontsize=14)
    axes[1].set_xlabel('Total Decrease Duration (days)', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)

    # Other details
    def round_up_to_nearest(value, factor):
        return np.ceil(value / factor) * factor

    max_duration = max(bins1.max(), bins2.max())
    max_frequency = max(hist1.max(), hist2.max())
    max_duration_rounded = round_up_to_nearest(max_duration, 50000)
    max_frequency_rounded = round_up_to_nearest(max_frequency, 5) + 5

    # Adjust the axis ranges across both subplots based on the analyte with higher values
    axes[0].set_xlim(0, max_duration_rounded)
    axes[1].set_xlim(0, max_duration_rounded)
    axes[0].set_ylim(0, max_frequency_rounded)
    axes[1].set_ylim(0, max_frequency_rounded)

    plt.tight_layout()
    plt.show()


In [ ]:
compare_analytes_histogram(sr_90_data_frame, i_129_data_frame, 'STRONTIUM-90', 'IODINE-129')

In [ ]:
def plot_log_time_to_MCL(df, analyte_name, font_size=14, title_font_size=18):
    valid_mcl_data = df[df['total_decrease_duration'].notnull()]
    valid_mcl_data = valid_mcl_data[valid_mcl_data['total_decrease_duration'] > 0]

    # Compute the log(total_decrease_duration)
    valid_mcl_data['log_time_to_MCL'] = np.log(valid_mcl_data['total_decrease_duration'])

    #Plotting
    plt.style.use(['science', 'no-latex'])  # Ensure SciencePlots is used
    plt.figure(figsize=(10, 6))

    plt.hist(valid_mcl_data['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)

    plt.title(f'Log(Time-to-MCL) Distribution for {analyte_name}', fontsize=title_font_size)
    plt.xlabel('Log(Time-to-MCL in days)', fontsize=font_size)
    plt.ylabel('Frequency', fontsize=font_size)

    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)

    plt.grid(False)
    plt.show()

    return valid_mcl_data[['well_name', 'total_decrease_duration', 'log_time_to_MCL']]

In [ ]:
log_time_to_mcl_sr_90 = plot_log_time_to_MCL(sr_90_data_frame, analyte_name='STRONTIUM-90', font_size=14, title_font_size=18)
print(log_time_to_mcl_sr_90)

In [ ]:
log_time_to_mcl_i_129 = plot_log_time_to_MCL(i_129_data_frame, analyte_name='IODINE-129', font_size=14, title_font_size=18)
print(log_time_to_mcl_i_129)

In [ ]:
def compare_log_time_to_MCL(df1, analyte1_name, df2, analyte2_name, font_size=14, title_font_size=18):
    valid_mcl_data1 = df1[df1['total_decrease_duration'].notnull() & (df1['total_decrease_duration'] > 0)]
    valid_mcl_data2 = df2[df2['total_decrease_duration'].notnull() & (df2['total_decrease_duration'] > 0)]

    # Compute the log(total_decrease_duration) for both analytes
    valid_mcl_data1['log_time_to_MCL'] = np.log(valid_mcl_data1['total_decrease_duration'])
    valid_mcl_data2['log_time_to_MCL'] = np.log(valid_mcl_data2['total_decrease_duration'])

    # Create vertical subplots
    plt.style.use(['science', 'no-latex'])  # Ensure SciencePlots is used
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))  # Vertical alignment

    # Function to round up to nearest specified factor
    def round_up_to_nearest(value, factor):
        return np.ceil(value / factor) * factor

    # Plotting
    hist1, bins1, _ = axes[0].hist(valid_mcl_data1['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)
    hist2, bins2, _ = axes[1].hist(valid_mcl_data2['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)

    axes[0].set_title(f'(a)', fontsize=title_font_size)
    axes[1].set_title(f'(b)', fontsize=title_font_size)
    axes[0].set_xlabel('Log(Time-to-MCL in days)', fontsize=font_size)
    axes[1].set_xlabel('Log(Time-to-MCL in days)', fontsize=font_size)
    axes[0].set_ylabel('Frequency', fontsize=font_size)
    axes[1].set_ylabel('Frequency', fontsize=font_size)

    axes[0].tick_params(axis='both', which='major', labelsize=font_size)
    axes[1].tick_params(axis='both', which='major', labelsize=font_size)

    max_duration = max(bins1.max(), bins2.max())
    max_frequency = max(hist1.max(), hist2.max())

    max_duration_rounded = round_up_to_nearest(max_duration, 0.5)  # Adjust for log-scale bins
    max_frequency_rounded = round_up_to_nearest(max_frequency, 5) + 2.5  # Add 2.5 as a buffer

    axes[0].set_xlim(0, max_duration_rounded)
    axes[1].set_xlim(0, max_duration_rounded)
    axes[0].set_ylim(0, max_frequency_rounded)
    axes[1].set_ylim(0, max_frequency_rounded)

    plt.tight_layout()
    plt.show()

In [ ]:
log_time_to_mcl_comparative = compare_log_time_to_MCL(sr_90_data_frame, 'STRONTIUM-90', i_129_data_frame, 'IODINE-129', font_size=14, title_font_size=18)


# **Mapping**

In [ ]:
import folium
import branca.colormap as cm
from matplotlib import cm, colors as mcolors
from folium.plugins import MeasureControl
from geopy.distance import geodesic

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
location_analyzer = DataAnalyzer('/content/gdrive/MyDrive/MIT Project - VAL @HW_CEE/FASB Well Construction Info.xlsx')

In [ ]:
location_analyzer.print_column_names()

In [ ]:
def calculate_real_distances(min_latitude, max_latitude, min_longitude, max_longitude):
    width = geodesic((min_latitude, min_longitude), (min_latitude, max_longitude)).meters
    height = geodesic((min_latitude, min_longitude), (max_latitude, min_longitude)).meters
    return width, height

In [ ]:
def generate_map(df, duration_column, zoom_level_offset=10, satellite_mode=False, aquifer=None):
    if aquifer:
        df = df[df['aquifer'] == aquifer]

    if df.empty:
        print(f"No wells found for the aquifer: {aquifer}")
        return None

    # Center of the bounding box
    center_latitude = df['latitude'].mean()
    center_longitude = df['longitude'].mean()

    # Zoom level
    max_latitude = df['latitude'].max()
    min_latitude = df['latitude'].min()
    max_longitude = df['longitude'].max()
    min_longitude = df['longitude'].min()
    max_delta = max(max_latitude - min_latitude, max_longitude - min_longitude)
    zoom_level = zoom_level_offset - int(np.log2(max_delta))  # Use the provided offset

    # Determine map type (satellite or normal)
    if satellite_mode:
        m = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_level, tiles=None)
        folium.TileLayer(
            tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            attr='Google',
            name='Google Satellite',
            max_zoom=20,
            subdomains=['mt0', 'mt1', 'mt2', 'mt3']
        ).add_to(m)
    else:
        m = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_level)

    conc_fg = folium.FeatureGroup(name="Concentrations")

    cmap_jet = cm.get_cmap('jet')
    num_colors = 6  # Optional number of colors
    color_list = [mcolors.rgb2hex(cmap_jet(i / num_colors)) for i in range(num_colors)]

    df['duration_years'] = df[duration_column] / 365.25

    df['log_duration_years'] = np.log(df['duration_years'])

    min_val_log = df['log_duration_years'].min()
    max_val_log = df['log_duration_years'].max()
    df['normalized_log_duration'] = (df['log_duration_years'] - min_val_log) / (max_val_log - min_val_log)

    cmap = mcolors.LinearSegmentedColormap.from_list('custom_colormap', color_list)

    for _, row in df.iterrows():
        coordinates = [row['latitude'], row['longitude']]
        point_color = mcolors.rgb2hex(cmap(row['normalized_log_duration']))

        # Creating popup content with specified information
        popup_content = f"<div style='max-width:800px;'><ul>"
        popup_content += f"<li>Well Name: {row['well_name']}</li>"
        popup_content += f"<li>Time to MCL: {row['total_decrease_duration']} days ({row['duration_years']:.2f} Years)</li>"
        popup_content += f"<li>Aquifer: {row['aquifer']}</li>"
        popup_content += "</ul></div>"

        folium.Circle(
            radius=15,
            location=coordinates,
            popup=folium.Popup(popup_content, max_width=800),
            fill_color=point_color,
            color='#000000',
            fill_opacity=1,
            stroke=1,
            weight=1
        ).add_to(conc_fg)

    m.add_child(conc_fg)

    min_val = df[duration_column].min()
    max_val = df[duration_column].max()
    min_val_years = min_val / 365.25
    max_val_years = max_val / 365.25

    colormap_original = folium.LinearColormap(colors=color_list, vmin=min_val_years, vmax=max_val_years, caption=f"{duration_column} (years)")
    colormap_original.add_to(m)

    width_meters, height_meters = calculate_real_distances(min_latitude, max_latitude, min_longitude, max_longitude)

    folium.Marker(
        [min_latitude, (min_longitude + max_longitude) / 2],
        icon=folium.DivIcon(html=f"<div style='font-size: 12pt'>Width: {width_meters:.2f} meters</div>")
    ).add_to(m)
    folium.Marker(
        [(min_latitude + max_latitude) / 2, min_longitude],
        icon=folium.DivIcon(html=f"<div style='font-size: 12pt; transform: rotate(90deg)'>Height: {height_meters:.2f} meters</div>")
    ).add_to(m)

    bounding_box = [
        [min_latitude, min_longitude],
        [min_latitude, max_longitude],
        [max_latitude, max_longitude],
        [max_latitude, min_longitude],
        [min_latitude, min_longitude]
    ]
    folium.PolyLine(bounding_box, color='black', weight=2, opacity=1).add_to(m)

    # Add vertical and horizontal lines of the map window
    top_horizontal_line = [
        [max_latitude, min_longitude],
        [max_latitude, max_longitude]
    ]
    bottom_horizontal_line = [
        [min_latitude, min_longitude],
        [min_latitude, max_longitude]
    ]
    left_vertical_line = [
        [min_latitude, min_longitude],
        [max_latitude, min_longitude]
    ]
    right_vertical_line = [
        [min_latitude, max_longitude],
        [max_latitude, max_longitude]
    ]

    folium.PolyLine(top_horizontal_line, color='blue', weight=2, opacity=1).add_to(m)
    folium.PolyLine(bottom_horizontal_line, color='blue', weight=2, opacity=1).add_to(m)
    folium.PolyLine(left_vertical_line, color='blue', weight=2, opacity=1).add_to(m)
    folium.PolyLine(right_vertical_line, color='blue', weight=2, opacity=1).add_to(m)

    return m

## **Sr-90**

In [ ]:
# DataFrame with decreasing trends
decreasing_well_list = data_analyzer.generate_decreasing_well_list(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='STRONTIUM-90',
    well_name_column='STATION_ID',
    MCL=10
)

# Extract latitude and longitude
location_data = location_analyzer.extract_location_data(
    well_name_column='station_id',
    decreasing_well_list=decreasing_well_list
)

# Merge DataFrame with latitude and longitude information
merged_df_sr_90 = pd.merge(analyte_summary_df_sr_90, location_data, left_on='well_name', right_on='station_id', how='inner')

print(merged_df_sr_90)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration')

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', zoom_level_offset=11)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', satellite_mode=True)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', aquifer='UAZ_UTRAU')

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', aquifer='LAZ_UTRAU')

## **I-129**

In [ ]:
# DataFrame with decreasing trends
decreasing_well_list = data_analyzer.generate_decreasing_well_list(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='IODINE-129',
    well_name_column='STATION_ID',
    MCL=10
)

# Extract latitude and longitude
location_data = location_analyzer.extract_location_data(
    well_name_column='station_id',
    decreasing_well_list=decreasing_well_list
)

# Merge DataFrame with latitude and longitude information
merged_df_i_129 = pd.merge(analyte_summary_df_i_129, location_data, left_on='well_name', right_on='station_id', how='inner')

print(merged_df_i_129)

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', zoom_level_offset=9)

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', satellite_mode=True)

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', aquifer='UAZ_UTRAU')

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', aquifer='LAZ_UTRAU')